In [ ]:
'''Reading Data'''

In [ ]:
!pip install mne
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt


     |████████████████████████████████| 7.4 MB 4.3 MB/s 


In [16]:
allfile_path=glob('dataset/*.edf')
allfile_path[0]

'dataset/s05.edf'

In [18]:
healthy_path=[i for i in allfile_path if 'h' in i.split('/')[1]]
patient_path=[i for i in allfile_path if 's' in i.split('/')[1]]

In [20]:
print(len(patient_path))

14


In [24]:
def read_data(file_path):
  data=mne.io.read_raw_edf(file_path, preload=True) #read data from file path
  data.set_eeg_reference() #by default takes average of all channels
  data.filter(l_freq=0.5,h_freq=45) # to be changed with bandpass filter?
  #segmentation
  epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1) #overlapping segments
  array=epochs.get_data() #converts mne epoch object to numpy array
  return(array)

In [26]:
sample_data=read_data(healthy_path[0])

Extracting EDF parameters from /content/dataset/h10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 278749  =      0.000 ...  1114.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 sec)

Not setting metadata
Not setting metadata
278 matching events found
No baseline correction applied
0 projection items activa

In [27]:
sample_data.shape # dimension of data => no.of epochs, channels , length of signal


(278, 19, 1250)

In [29]:
%%capture 
#capture is used to prevent the data jargon from printing
healthy_epochs_array=[read_data(i) for i in healthy_path]
patient_epochs_array=[read_data(i) for i in patient_path]

In [32]:
#creating labels 0 for healthy and 1 for patient
healthy_epoch_labels=[len(i)*[0] for i in healthy_epochs_array]
patient_epoch_labels=[len(i)*[0] for i in patient_epochs_array]

In [35]:
data_list=healthy_epochs_array+patient_epochs_array
label_list=healthy_epoch_labels+patient_epoch_labels

In [39]:
%%capture
#splitting 
group_list=[[i]*len(j) for i,j in enumerate(data_list)] #list of 0 for element 1 list of 1 for next etc multiplied bt length
group_list

In [42]:
data_array=np.vstack(data_list) #stack arrays vertically
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)

#7201 epochs of length 1250

In [ ]:
'''Feature Extaction'''

In [ ]:
'''Convolution'''